Utilities for working with PIL, tensor, numpy and imgaug images

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from scipy.linalg import toeplitz

In [2]:
# Define variables

# Image
img = torch.tensor([[1, 3, 1, 2],
                    [0, 2, 1, 4],
                    [2, 1, 0, 1],
                    [3, 1, 2, 3]]).float()

# Filter
w = torch.tensor([[1, 1, 1],
                  [1, 1, 1],
                  [1, 1, 1]]).float()

padding = 1
kernel_size = w.shape[0]

In [3]:
# Convolution using Pytorch

conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=kernel_size, stride=1, padding=padding, 
                   bias=False, padding_mode='zeros')

with torch.no_grad():
    # Change values in place 
    #conv2d.weight[:] = w.view(1, 1, 3, 3)
    # Or add values as parameter
    conv2d.weight = nn.Parameter(w.view(1, 1, 3, 3))
    
res_conv2d = conv2d(img[None,None])
print(res_conv2d)

tensor([[[[ 6.,  8., 13.,  8.],
          [ 9., 11., 15.,  9.],
          [ 9., 12., 15., 11.],
          [ 7.,  9.,  8.,  6.]]]], grad_fn=<ThnnConv2DBackward>)


In [81]:
# Convolution by unfolding each image block into one column. For instance, image
#|1 2 3|
#|4 5 6|
#|7 8 9|
# with a 2x2 kernel becomes
#|1 2 4 5|  
#|2 3 5 6|
#|4 5 7 8|
#|5 6 8 9|
# Then, convolution becomes a matrix multiplication between the unfolded image and a flattened kernel

img_unf = nn.functional.unfold(img[None,None,...], (3, 3), padding=1).transpose(1, 2)
out_unf = img_unf.matmul(w.view(-1,1))
#out = nn.functional.fold(out_unf, (4, 4), (1, 1))  # alternative
res_unfold = out_unf.view(1, 1, 4, 4)
print(res_unfold)

tensor([[[[ 6.,  8., 13.,  8.],
          [ 9., 11., 15.,  9.],
          [ 9., 12., 15., 11.],
          [ 7.,  9.,  8.,  6.]]]])


In [48]:
# Convolution by creating a circulant matrix using the filter values
# for a 2x2 filter
#|1 2|
#|3 4|
#and a 3x3 image, the filter becomes
#|1 2 0 3 4 0 0 0 0|
#|0 1 2 0 3 4 0 0 0|
#|0 0 0 1 2 0 3 4 0|
#|0 0 0 0 1 2 0 3 4|
# The, convolution becomes a matrix multiplication between this matrix and the flattened image

H = img.shape[0]
W = img.shape[1]
K = kernel_size
num_blocks_h = (H+2*padding-K+1)  # Number of filter positions in the vertical direction
num_blocks_w = (W+2*padding-K+1)  # Number of filter positions in the horizontal direction
num_blocks = num_blocks_h*num_blocks_w  # Number of filter positions

zero_length = W+2*padding-K       # Number of zeros between rows of filter values
first_row = torch.cat((w, torch.zeros(K,zero_length)), axis=1).view(1, -1)
first_row = torch.cat((first_row, torch.zeros(1,(H+2*padding-K)*(W+2*padding))), axis=1)

w_matrix = torch.zeros(num_blocks, first_row.numel())
for i in range(num_blocks):
    w_matrix[i] = first_row
    if (i+1)%num_blocks_w==0:  # If at the end of image row
        first_row = first_row.roll(K)
    else:
        first_row = first_row.roll(1)
        
img_column = nn.functional.pad(img, (padding, padding, padding, padding)).view(-1,1)
res_filt_mat = w_matrix.matmul(img_column).view(img.shape)
print(res_filt_mat)

tensor([[ 6.,  8., 13.,  8.],
        [ 9., 11., 15.,  9.],
        [ 9., 12., 15., 11.],
        [ 7.,  9.,  8.,  6.]])


In [72]:
# Calculate the gradient of the output with respect to the filter weights
output = res_conv2d.sum()
output.backward()
gradients = conv2d.weight.grad
print(gradients)

tensor([[[[11., 18., 15.],
          [17., 27., 21.],
          [12., 20., 15.]]]])


Let $\vec{x}$ represent the values of the padded image, $\vec{y}$ represent the result of the convolution, $\vec{w}$ the values of the filter and $o$ the `output` variable. We need to calculate the gradient of $o$ with respect to each weight of the filter, that is, we need
\begin{equation}
\frac{\partial o}{\partial \vec{w}}
\end{equation}

This is given by

\begin{equation}
\frac{\partial o}{\partial \vec{w}} = \frac{\partial o}{\partial \vec{y}} \frac{\partial \vec{y}}{\partial \vec{w}}
\end{equation}

$\frac{\partial o}{\partial \vec{y}}$ is the gradient of $o$ with respect to the output image. $\frac{\partial \vec{y}}{\partial \vec{w}}$ is the Jacobian matrix of the output with respect to the weights.

As an ilustration, let's calculate $\frac{\partial o}{\partial w_1}$. This is given by

\begin{equation}
\frac{\partial o}{\partial w_1} = \frac{\partial o}{\partial \vec{y}} \frac{\partial \vec{y}}{\partial w_1}
\end{equation}

Since 

\begin{equation}
o = \sum y_i
\end{equation}
we have

\begin{align}
\frac{\partial o}{\partial \vec{y}} & = \left( \frac{\partial o}{\partial y_1},\frac{\partial o}{\partial y_2},\dots,\frac{\partial o}{\partial y_n} \right) \\
& = \left( 1, 1, \dots, 1 \right).
\end{align}

For the other term of the equation, we have

\begin{align}
\frac{\partial \vec{y}}{\partial w_1} & = \left( \frac{\partial y_1}{\partial w_1},\frac{\partial y_2}{\partial w_1},\dots,\frac{\partial y_n}{\partial w_1} \right).
\end{align}
For instance, $y_6$ (the value at row 2 column 2 of the result) is calculated as

\begin{equation}
y_6 = x_8*w_1 + x_9*w_2 + x_{10}*w_3 + x_{14}*w_4 + x_{15}*w_5 + x_{16}*w_6 + x_{20}*w_7 + x_{21}*w_8 + x_{22}*w_9
\end{equation}
in the convolution, and therefore

\begin{equation}
\frac{\partial y_6}{\partial w_1} = x_8.
\end{equation}

Notice that in our notation $x_8$ corresponds to the value at row 1 and column 1 of the original image since the calculations involve some padding in the original image. Calculating the remaining values we have

\begin{equation}
\frac{\partial \vec{y}}{\partial w_1} = \left( x_1, x_2, x_3, x_4, x_7, x_8, x_9, x_{10}, x_{13}, x_{14}, x_{15}, x_{16}, x_{19}, x_{20}, x_{21}, x_{22} \right).
\end{equation}
Since we padded the image with zeros

\begin{equation}
\frac{\partial \vec{y}}{\partial w_1} = \left( 0, 0, 0, 0, 0, x_8, x_9, x_{10}, 0, x_{14}, x_{15}, x_{16}, 0, x_{20}, x_{21}, x_{22} \right).
\end{equation}
Thus

\begin{equation}
\frac{\partial o}{\partial w_1} = x_8 + x_9 + x_{10} + x_{14} + x_{15} + x_{16} + x_{20} + x_{21} + x_{22}.
\end{equation}

The full Jacobian matrix is given by

\begin{equation}
J = 
\begin{pmatrix}
\frac{\partial y_1}{\partial w_1}    & \frac{\partial y_1}{\partial w_2}    & \dots  & \frac{\partial y_1}{\partial w_9} \\
\frac{\partial y_2}{\partial w_1}    & \frac{\partial y_2}{\partial w_2}    & \dots  & \frac{\partial y_2}{\partial w_9} \\
\vdots                               &                            \ddots    & \ddots & \vdots \\
\frac{\partial y_{16}}{\partial w_1} & \frac{\partial y_{16}}{\partial w_2} & \dots  & \frac{\partial y_{16}}{\partial w_9}
\end{pmatrix}
\end{equation}

\begin{equation}
J = 
\begin{pmatrix}
x_1    & x_2    & x_3    & x_7    & x_8    & x_9    & x_{13} & x_{14} & x_{15} \\
x_2    & x_3    & x_4    & x_8    & x_9    & x_{10} & x_{14} & x_{15} & x_{16} \\
x_3    & x_4    & x_5    & x_9    & x_{10} & x_{11} & x_{15} & x_{16} & x_{17} \\
x_4    & x_5    & x_6    & x_{10} & x_{11} & x_{12} & x_{16} & x_{17} & x_{18} \\
x_7    & x_8    & x_9    & x_{13} & x_{14} & x_{15} & x_{19} & x_{20} & x_{21} \\
x_8    & x_9    & x_{10} & x_{14} & x_{15} & x_{16} & x_{20} & x_{21} & x_{22} \\
x_9    & x_{10} & x_{11} & x_{15} & x_{16} & x_{17} & x_{21} & x_{22} & x_{23} \\
x_{10} & x_{11} & x_{12} & x_{16} & x_{17} & x_{18} & x_{22} & x_{23} & x_{24} \\
x_{13} & x_{14} & x_{15} & x_{19} & x_{20} & x_{21} & x_{25} & x_{26} & x_{27} \\
x_{14} & x_{15} & x_{16} & x_{20} & x_{21} & x_{22} & x_{26} & x_{27} & x_{28} \\
x_{15} & x_{16} & x_{17} & x_{21} & x_{22} & x_{23} & x_{27} & x_{28} & x_{29} \\
x_{16} & x_{17} & x_{18} & x_{22} & x_{23} & x_{24} & x_{28} & x_{29} & x_{30} \\
x_{19} & x_{20} & x_{21} & x_{25} & x_{26} & x_{27} & x_{31} & x_{32} & x_{33} \\
x_{20} & x_{21} & x_{22} & x_{26} & x_{27} & x_{28} & x_{32} & x_{33} & x_{34} \\
x_{21} & x_{22} & x_{23} & x_{27} & x_{28} & x_{29} & x_{33} & x_{34} & x_{35} \\
x_{22} & x_{23} & x_{24} & x_{28} & x_{29} & x_{30} & x_{34} & x_{35} & x_{36} \\
\end{pmatrix}
\end{equation}

But this matrix is exactly the same as `img_unf`! Thus, for convolving an image with a filter we can do

\begin{equation}
\vec{y} = J\vec{w}
\end{equation}
and for calculating the gradients of the output with respect to the weights we can do

\begin{equation}
\frac{\partial o}{\partial \vec{w}} = J^T\frac{\partial o}{\partial \vec{y}}
\end{equation}

In [83]:
gradients_matmul = img_unf.squeeze().T.matmul(torch.ones(res_conv2d.numel())).view(w.shape)
print((gradients_matmul-gradients.squeeze()).abs().sum())

tensor(0.)


In [118]:
# Implementing a conv2d autograd function

class conv2d(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, weight, padding):
        ctx.save_for_backward(input, weight)
        ctx.padding = padding
        img_unf = nn.functional.unfold(input[None,None,...], weight.shape, padding=padding).transpose(1, 2)
        output = img_unf.matmul(weight.view(-1,1))
        return output.view(input.shape[0]+2*padding-weight.shape[0]+1, input.shape[1]+2*padding-weight.shape[1]+1)

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        input, weight = ctx.saved_tensors
        padding = ctx.padding
        img_unf = nn.functional.unfold(input[None,None,...], weight.shape, padding=padding).transpose(1, 2)
        gradients_matmul = img_unf.squeeze().T.matmul(grad_output.view(-1,1)).view(weight.shape)
        #gradients_matmul = torch.ones(weight.shape)
        return None, gradients_matmul, None
    
w2 = nn.Parameter(w)
res = conv2d.apply(img, w2, 1)
s = res.sum()
s.backward()

print(w2.grad)

tensor([[11., 18., 15.],
        [17., 27., 21.],
        [12., 20., 15.]])
